Annie
====

### 통계기반 한국어 개체명 인식기

* 2016년 10월 7일
* 임재수 <krikit@naver.com>
* https://github.com/krikit/annie

<img src="https://encrypted-tbn3.gstatic.com/images?q=tbn:ANd9GcRVT1pmeWfu_BK832v-DP2jdLj5hcuyE1WmFQVlQeULyDc5FTOq" width=300 />


목차
----
1. 개체명 인식 개요
2. 사전 매칭 (Baseline)
3. CRF 기반 개체명 인식기
4. SVM 기반 인명 분류기
5. 성능 평가

1. 개체명 인식 개요
----

### [IOB2 태깅](https://en.wikipedia.org/wiki/Inside_Outside_Beginning)

```
The IOB format (short for Inside, Outside, Beginning) is a common tagging format for tagging tokens in a chunking task in computational linguistics (ex. Named Entity Recognition).
-- from Wikipedia
```

* 예

| 형태소  | 태그 |
|--------|-----|
| 박항서  | B-PS |
| 전남    | B-OG |
| 드래곤즈 | I-OG |
| 감독    | O    |


* 개체명을 인식하는 문제
    - <img src="img/arrow_down.png" style="height: 20px" align="left" />
* 개체명의 경계와 종류를 결정하는 문제
    - <img src="img/arrow_down.png" style="height: 20px" align="left" />
* 각 형태소의 IOB2 태그를 분류하는 문제


<img src="http://core0.staticworld.net/images/article/2014/04/blank-tag-isolated-on-white-95754104-100264644-primary.idge.jpg" width=300 />

2. 사전 매칭 (Baseline)
----

### 사전 구축

* 코퍼스와 함께 배포한 사전(gazette) + 학습 코퍼스(train.json)으로부터 추출한 사전
    - <img src="img/arrow_down.png" style="height: 20px" align="left" />
* gazette.annie


* 동일한 표제어에 대해 카테고리가 여럿인 경우
    - <img src="img/arrow_down.png" style="height: 20px" align="left" />
* 빈도가 높은 순으로 정렬하여 저장

```
맨체스터    LC,OG
전주    LC,OG,DT
```

### 날짜/시간 정규화

* 날짜(DT)와 시간(TI)의 경우 숫자를 많이 사용
    - <img src="img/arrow_down.png" style="height: 20px" align="left" />
* 숫자를 그대로 매칭하면 히트율이 낮음
    - <img src="img/arrow_down.png" style="height: 20px" align="left" />
* 숫자를 모두 0으로 정규화

```
2016년 10월 7일  =>  0000년 00년 0일
```

* 사전을 정규화하여 저장, 입력 문장도 정규화한 다음, 날짜/시간 표현을 우선적으로 매칭 시도

### Baseline 시스템

* 사전 매칭한 다음 가장 빈도가 높은 카테고리를 선택
* 1음절 개체명은 제거


* 성능 개요 (dev 코퍼스)
    - Precision: 0.7129
    - Recall:    0.7174
    - F1-score:  0.7152


* 카테고리별 성능 상세

```
======== DT ========
# of NEs in gold standard: 316
# of NEs in test file    : 386
# of NEs in both(matched): 246
Precision: 0.6373
Recall:    0.7785
F1-score:  0.7009

======== LC ========
# of NEs in gold standard: 238
# of NEs in test file    : 359
# of NEs in both(matched): 209
Precision: 0.5822
Recall:    0.8782
F1-score:  0.7002

======== OG ========
# of NEs in gold standard: 412
# of NEs in test file    : 383
# of NEs in both(matched): 292
Precision: 0.7624
Recall:    0.7087
F1-score:  0.7346

======== PS ========
# of NEs in gold standard: 581
# of NEs in test file    : 403
# of NEs in both(matched): 356
Precision: 0.8834
Recall:    0.6127
F1-score:  0.7236

======== TI ========
# of NEs in gold standard: 42
# of NEs in test file    : 68
# of NEs in both(matched): 37
Precision: 0.5441
Recall:    0.8810
F1-score:  0.6727
```

### 관련 코드

* 사전 핸들링 라이브러리: [lib/gazette.py](https://github.com/krikit/annie/blob/master/lib/gazette.py)
* 사전 구축 프로그램: [bin/build_gazette.py](https://github.com/krikit/annie/blob/master/bin/build_gazette.py)
* 베이스라인 태깅 프로그램: [bin/baseline.py](https://github.com/krikit/annie/blob/master/bin/baseline.py)


<img src="https://lh4.ggpht.com/rLVQGRVlKPEPlzMh-Vp6wHr6y6ELJdCTpxENwt_TKBNDw_pReiCAwDvE27t-jlTJfw=w170" width=200 />

3. CRF 기반 개체명 인식기
----

### 자질

* 좌/우 주변 -2 ~ +2 위치의 5개 형태소 기반
    - 형태소, 형태소 bigram
    - 품사 태그 및 그 bigram, trigram
    - gazzete 사전 매칭 IOB2 태그 및 그 bigram, trigram
    - 1, 2음절 prefix/suffix
    - 형태소의 길이 (단독으로 사용하지 않고 아래의 조합만 사용)
        * gazette 사전 매칭 + 길이
        * 1, 2음절 prefix/suffix + 길이
    - lexical form(pattern)
        * 한글 -> '가', 한자 -> '漢', 영문 -> 'A', 숫자 -> '0', 기호 -> '.'
        ```
        박근혜  =>  가가가
        Obama  =>  AAAAA
        2016년  => 0000가
        지이언-T  =>  가가가.A
        ```
    - begin/end of sentence  =>  BOS, EOS
    - begin/middle/end of word(어절)  =>  BOW, MOW, EOW
    - 이전 어절의 마지막 형태소, 다음 어절의 첫 형태소 및 그 결합

### 성능

* 성능 개요 (dev 코퍼스)
    - Precision: 0.8816
    - Recall:    0.7495
    - F1-score:  0.8102


* 카테고리별 성능 상세

```
======== DT ========
# of NEs in gold standard: 316
# of NEs in test file    : 304
# of NEs in both(matched): 276
Precision: 0.9079
Recall:    0.8734
F1-score:  0.8903

======== LC ========
# of NEs in gold standard: 238
# of NEs in test file    : 285
# of NEs in both(matched): 216
Precision: 0.7579
Recall:    0.9076
F1-score:  0.8260

======== OG ========
# of NEs in gold standard: 412
# of NEs in test file    : 327
# of NEs in both(matched): 285
Precision: 0.8716
Recall:    0.6917
F1-score:  0.7713

======== PS ========
# of NEs in gold standard: 581
# of NEs in test file    : 393
# of NEs in both(matched): 376
Precision: 0.9567
Recall:    0.6472
F1-score:  0.7721

======== TI ========
# of NEs in gold standard: 42
# of NEs in test file    : 42
# of NEs in both(matched): 38
Precision: 0.9048
Recall:    0.9048
F1-score:  0.9048
```

### 분석

* 개체명 중 인명(PS)의 비율이 높은 데 반해 시스템의 재현율이 가장 낮음
    - <img src="img/arrow_down.png" style="height: 20px" align="left" />
* 인명을 별도로 인식하는 후처리 분류기
* 실험 노트
    - 인명 후보 Histogram: [NNP+NNG_2+3](NNP+NNG_2+3.ipynb)
    - 인명 Histogram: [ps_histogram](ps_histogram.ipynb)

### 관련 코드

* 자질 추출 라이브러리: [lib/feature.py](https://github.com/krikit/annie/blob/master/lib/feature.py)
* JSON 입력을 자질 형태로 변환 프로그램: [bin/json2feat.py](https://github.com/krikit/annie/blob/master/bin/json2feat.py)
* CRFsuite IOB 태깅 결과를 JSON으로 변환 프로그램: [bin/iob2json.py](https://github.com/krikit/annie/blob/master/bin/iob2json.py)


<img src="https://folio.openknowl.com/old/image/a91423e0146427b339968cee1e576b47.jpg" width=300 />

4. SVM 기반 인명 분류기
----

### 적용 범위

* CRF 태깅 결과 개체명이 아닌 단일 형태소 중 3음절의 NNP 품사를 갖는 형태소에 대해 수행
    - <img src="img/arrow_down.png" style="height: 20px" align="left" />
* B-PS 혹은 O를 판단하는 binary 분류기

### 자질

* 코퍼스와 함께 배포된 pretrained word2vec 사용
    - skip-gram, window 5, 50차원
    - 조사를 포함하는 형태소 단위
    - key: '형태소/품사태그' 조합
    - 숫자는 'NUM/SN'으로 정규화


* 인명을 분류하려는 형태소 및 좌우 2개의 형태소 (250차원)


* OOV(out of vocabulary) 형태소의 처리
    - 동일 품사를 갖는 모든 벡터의 centroid(평균 벡터)를 사용

### 학습

* train 코퍼스에서 3음절이며 NNP 품사를 갖는 모든 형태소 중
    - 단일 형태소가 인명인 경우 positive: 1,488건
    - 여러 형태소이거나 인명이 아닌 경우 negative: 1,420건


* dev 코퍼스에 적용한 적확도: 0.8964


* 실험 노트
    - 인명 분류기: [ps_classifier](ps_classifier.ipynb)

### 성능

* 전체 성능 개선 (dev 코퍼스)
    - Precision: 0.8816  =>  0.8753  (-0.0063)
    - Recall:    0.7495  =>  0.8043  (+0.0548)
    - F1-score:  0.8102  =>  0.8383  (+0.0281)


* 인명 성능 개선
    - Precision: 0.9567  =>  0.9223  (-0.0344)
    - Recall:    0.6472  =>  0.7969  (+0.1497)
    - F1-score:  0.7721  =>  0.8550  (+0.0828)

### 관련 코드

* word2vec 관련 라이브러리: [lib/word2vec.py](https://github.com/krikit/annie/blob/master/lib/word2vec.py)
* 인명 추가 태깅 프로그램: [bin/tag_ps.py](https://github.com/krikit/annie/blob/master/bin/tag_ps.py)


<img src="http://webimage.10x10.co.kr/image/main/21/M000214559.jpg" width=300 />

5. 성능 평가
----

### test 코퍼스

* NEtaggedCorpus_test.json

* 성능 개요
    - Precision: 0.8631
    - Recall:    0.8213
    - F1-score:  0.8417


* 카테고리별 성능 상세

```
======== DT ========
# of NEs in gold standard: 463
# of NEs in test file    : 465
# of NEs in both(matched): 401
Precision: 0.8624
Recall:    0.8661
F1-score:  0.8642

======== LC ========
# of NEs in gold standard: 484
# of NEs in test file    : 527
# of NEs in both(matched): 425
Precision: 0.8065
Recall:    0.8781
F1-score:  0.8408

======== OG ========
# of NEs in gold standard: 701
# of NEs in test file    : 582
# of NEs in both(matched): 506
Precision: 0.8694
Recall:    0.7218
F1-score:  0.7888

======== PS ========
# of NEs in gold standard: 907
# of NEs in test file    : 860
# of NEs in both(matched): 766
Precision: 0.8907
Recall:    0.8445
F1-score:  0.8670

======== TI ========
# of NEs in gold standard: 86
# of NEs in test file    : 79
# of NEs in both(matched): 71
Precision: 0.8987
Recall:    0.8256
F1-score:  0.8606
```

### 의문의(?) 코퍼스

* 2016klp_1000Sentences.json

* 성능 개요
    - Precision: 0.8186
    - Recall:    0.5159
    - F1-score:  0.6329


* 카테고리별 성능 상세

```
======== DT ========
# of NEs in gold standard: 602
# of NEs in test file    : 514
# of NEs in both(matched): 413
Precision: 0.8035
Recall:    0.6860
F1-score:  0.7401

======== LC ========
# of NEs in gold standard: 883
# of NEs in test file    : 613
# of NEs in both(matched): 579
Precision: 0.9445
Recall:    0.6557
F1-score:  0.7741

======== OG ========
# of NEs in gold standard: 678
# of NEs in test file    : 246
# of NEs in both(matched): 165
Precision: 0.6707
Recall:    0.2434
F1-score:  0.3571

======== PS ========
# of NEs in gold standard: 596
# of NEs in test file    : 339
# of NEs in both(matched): 252
Precision: 0.7434
Recall:    0.4228
F1-score:  0.5390
```


<img src="http://christianvisionalliance.org/wp-content/uploads/2016/06/thank-you-from-christian-vision-alliance.jpg" width=500 />